In [ ]:
#Name:Yen Pham
#CSCE 5290  Project

import pandas as pd
import spacy
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Reddit_covid_threads.csv to Reddit_covid_threads.csv
Saving Twitter_covid_tweets.csv to Twitter_covid_tweets.csv
Saving Youtube_covid_comments.csv to Youtube_covid_comments.csv


In [ ]:
import io
# Read csv file to dataframes
df1 = pd.read_csv(io.BytesIO(uploaded['Reddit_covid_threads.csv']))
df2 = pd.read_csv(io.BytesIO(uploaded['Twitter_covid_tweets.csv']))
df3 = pd.read_csv(io.BytesIO(uploaded['Youtube_covid_comments.csv']))

df = pd.concat([df1,df2,df3])

In [ ]:
# Lemmatize text to get the original word meanings and take the nouns of them
def only_nouns(texts):
    output = []
    for doc in nlp.pipe(texts):
        noun_text = " ".join(token.lemma_ for token in doc if token.pos_ == 'NOUN')
        output.append(noun_text)
    return output


df['Text'] = only_nouns(df['Text'])

df.head()

,Created_at,User_ID,Text
0,2021-04-03 20:07:49,n3v3r0dd0r3v3n,link end chapter
1,2021-04-07 08:05:05,orangetato,opinion time point purpose
2,2021-04-21 11:34:13,SwinubIsDivinub,lockdown wing policy core
3,2021-05-06 17:37:43,Sandshrrew,idea sub people side mankind :) guy brother si...
4,2021-04-03 12:17:14,thinkinanddrinkin,body nature illness man doctor illness doctor ...


In [ ]:
# number of topics to extract
n_topics = 3
random_state = 0
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vec = TfidfVectorizer(max_features=5000, stop_words="english", max_df=0.95, min_df=2)
features = vec.fit_transform(df.Text)

# train dataset
from sklearn.decomposition import NMF
cls = NMF(n_components=n_topics, random_state=random_state)
cls.fit(features)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
    n_components=3, random_state=0, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

In [ ]:
# list of unique words found by the vectorizer
feature_names = vec.get_feature_names()

# number of most influencing words to display per topic
n_top_words = 30

for i, topic_vec in enumerate(cls.components_):
    print(i, end=' ')
    # topic_vec.argsort() produces a new array
    # in which word_index with the least score is the
    # first array element and word_index with highest
    # score is the last array element. Then using a
    # fancy indexing [-1: -n_top_words-1:-1], we are
    # slicing the array from its end in such a way that
    # top `n_top_words` word_index with highest scores
    # are returned in desceding order
    for fid in topic_vec.argsort()[-1:-n_top_words-1:-1]:
        print(feature_names[fid], end=' ')
    print()

0 people mask thing time virus life lot government year job way hospital health pandemic number country risk world point mandate vaccination state person reason family place immunity problem month shit 
1 case day death week test today number time rate dose covid year month hospitalization school population datum booster shot state yesterday hour symptom vaccination source positivity average age total news 
2 vaccine booster shot dose virus year mandate month child effect time kid immunity vaccination risk covid way flu mask infection passport country thing study health doctor antibody life work world 


In [ ]:
# Topic 0 is about government regulations about covid and people's personal situations/feelings during covid, such as wearing masks, lockdown, vaccination/health check madate, covid stimulus, school/work/job openings, restrictions, sickness, poverty, etc
# Topic 1 is about covid reports, such as weekly/daily death rate, infection rate, hositalization rate, covid variants, covid symptoms, average ages to get infected, etc.
# Topic 2 is about vaccination and human immunity & biology/medical facts - eiter natural immunity or through vaccination, as well as vaccine risks especially on kids.  

# for testing
testing_submissions = [
    "Except that's exactly how the right reacts to elderly people dying.",
    "They're so quick to hand wave potentially millions of people away like it's nothing.",
    "Also, feel free to link any study demonstrating the vaccines have caused an influx of deaths.",
    "We have 200,000 cases per day"
]
cls.transform(vec.transform(testing_submissions)).argsort(axis=1)[:,-1]

array([0, 0, 2, 1])

In [ ]:
# get the categories of 3 dataframes
df1['Category'] = cls.transform(vec.transform(df1['Text'].to_list())).argsort(axis=1)[:,-1]
df2['Category'] = cls.transform(vec.transform(df2['Text'].to_list())).argsort(axis=1)[:,-1]
df3['Category'] = cls.transform(vec.transform(df3['Text'].to_list())).argsort(axis=1)[:,-1]

In [ ]:
from google.colab import files

# put dataframes into files
df1.to_csv('Reddit_covid_threads.csv', index=False)
files.download('Reddit_covid_threads.csv')

df2.to_csv('Twitter_covid_tweets.csv', index=False)
files.download('Twitter_covid_tweets.csv')

df3.to_csv('Youtube_covid_comments.csv', index=False)
files.download('Youtube_covid_comments.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>